<a href="https://colab.research.google.com/github/kburke-code/Learning-Python/blob/main/Supervised%20LEarning%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Bayesian Network"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 2,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "0.0192\n",
      "0.011199999999999996\n"
     ]
    }
   ],
   "source": [
    "from pomegranate import *\n",
    "\n",
    "rain = Node(DiscreteDistribution({\n",
    "    \"none\": 0.7,\n",
    "    \"light\": 0.2,\n",
    "    \"heavy\": 0.1\n",
    "}), name=\"rain\")\n",
    "\n",
    "maintenance = Node(ConditionalProbabilityTable([\n",
    "    [\"none\", \"yes\", 0.4],\n",
    "    [\"none\", \"no\", 0.6],\n",
    "    [\"light\", \"yes\", 0.2],\n",
    "    [\"light\", \"no\", 0.8],\n",
    "    [\"heavy\", \"yes\", 0.1],\n",
    "    [\"heavy\", \"no\", 0.9]\n",
    "], [rain.distribution]), name=\"maintenance\")\n",
    "\n",
    "train = Node(ConditionalProbabilityTable([\n",
    "    [\"none\", \"yes\", \"on time\", 0.8],\n",
    "    [\"none\", \"yes\", \"delayed\", 0.2],\n",
    "    [\"none\", \"no\", \"on time\", 0.9],\n",
    "    [\"none\", \"no\", \"delayed\", 0.1],\n",
    "    [\"light\", \"yes\", \"on time\", 0.6],\n",
    "    [\"light\", \"yes\", \"delayed\", 0.4],\n",
    "    [\"light\", \"no\", \"on time\", 0.7],\n",
    "    [\"light\", \"no\", \"delayed\", 0.3],\n",
    "    [\"heavy\", \"yes\", \"on time\", 0.4],\n",
    "    [\"heavy\", \"yes\", \"delayed\", 0.6],\n",
    "    [\"heavy\", \"no\", \"on time\", 0.5],\n",
    "    [\"heavy\", \"no\", \"delayed\", 0.5],\n",
    "], [rain.distribution, maintenance.distribution]), name=\"train\")\n",
    "\n",
    "appointment = Node(ConditionalProbabilityTable([\n",
    "    [\"on time\", \"attend\", 0.9],\n",
    "    [\"on time\", \"miss\", 0.1],\n",
    "    [\"delayed\", \"attend\", 0.6],\n",
    "    [\"delayed\", \"miss\", 0.4]\n",
    "], [train.distribution]), name=\"appointment\")\n",
    "\n",
    "# An instance of a Bayesian Network\n",
    "model = BayesianNetwork()\n",
    "\n",
    "# Add states\n",
    "model.add_states(rain, maintenance, train, appointment)\n",
    "\n",
    "# Add edges connecting nodes\n",
    "model.add_edge(rain, maintenance)\n",
    "model.add_edge(rain, train)\n",
    "model.add_edge(maintenance, train)\n",
    "model.add_edge(train, appointment)\n",
    "\n",
    "# Finalize model\n",
    "model.bake()\n",
    "\n",
    "\n",
    "# Calculate probability for a given observation\n",
    "probability = model.probability([[\"light\", \"no\", \"delayed\", \"miss\"]])\n",
    "print(probability)\n",
    "\n",
    "\n",
    "# Calculate probability for a given observation\n",
    "probability = model.probability([[\"light\", \"no\", \"on time\", \"miss\"]])\n",
    "print(probability)\n",
    "\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### If rain = light, maintainance = no, appointment = miss; what is the chance that train has arriaved in time?"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 29,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "rain: light\n",
      "maintenance: no\n",
      "train\n",
      "    delayed: 0.6316\n",
      "    on time: 0.3684\n",
      "appointment: miss\n"
     ]
    }
   ],
   "source": [
    "         \n",
    "# Calculate predictions\n",
    "predictions = model.predict_proba({\"appointment\": \"miss\", \"rain\": \"light\", \"maintenance\":\"no\"})\n",
    "#predictions = model.predict_proba({\"train\": \"delayed\", \"rain\": \"light\"})\n",
    "\n",
    "#print (predictions)\n",
    "\n",
    "\n",
    "# Print predictions for each node\n",
    "for node, prediction in zip(model.states, predictions):\n",
    "    if isinstance(prediction, str):\n",
    "        print(f\"{node.name}: {prediction}\")\n",
    "    else:\n",
    "        print(f\"{node.name}\")\n",
    "        for value, probability in prediction.parameters[0].items():\n",
    "            print(f\"    {value}: {probability:.4f}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### If rain = light, maintainance = no, train= delayed; what is the chance that I will miss the appointment?"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 30,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "rain: light\n",
      "maintenance: no\n",
      "train: delayed\n",
      "appointment\n",
      "    attend: 0.6000\n",
      "    miss: 0.4000\n"
     ]
    }
   ],
   "source": [
    "# Calculate predictions\n",
    "predictions = model.predict_proba({\"train\": \"delayed\", \"rain\": \"light\", \"maintenance\":\"no\"})\n",
    "\n",
    "# Print predictions for each node\n",
    "for node, prediction in zip(model.states, predictions):\n",
    "    if isinstance(prediction, str):\n",
    "        print(f\"{node.name}: {prediction}\")\n",
    "    else:\n",
    "        print(f\"{node.name}\")\n",
    "        for value, probability in prediction.parameters[0].items():\n",
    "            print(f\"    {value}: {probability:.4f}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": []
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.5"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}


NameError: ignored